In [ ]:
from __future__ import print_function, division
from builtins import range
from future.utils import iteritems
from nltk.corpus import brown
import operator
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))

In [ ]:
KEEP_WORDS = set([
  'king', 'man', 'queen', 'woman',
  'italy', 'rome', 'france', 'paris',
  'london', 'britain', 'england',
])


def get_sentences():
    return brown.sents()

In [ ]:
def get_sentences_with_word2idx():
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START': 0, 'END': 1}
    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                word2idx[token] = i
                i += 1
            indexed_sentence.append(word2idx[token])
        indexed_sentences.append(indexed_sentence)
    print("Vocab size:", i)
    return indexed_sentences, word2idx

In [ ]:
def get_sentences_with_word2idx_limit_vocab(n_vocab=2000, keep_words=KEEP_WORDS):
    sentences = get_sentences()
    indexed_sentences = []
    i = 2
    word2idx = {'START': 0, 'END': 1}
    idx2word = ['START', 'END']

    word_idx_count = {0: float('inf'),1: float('inf')}

    for sentence in sentences:
        indexed_sentence = []
        for token in sentence:
            token = token.lower()
            if token not in word2idx:
                idx2word.append(token)
                word2idx[token] = i
                i += 1

            # keep track of counts for later sorting
            idx = word2idx[token]
            word_idx_count[idx] = word_idx_count.get(idx, 0) + 1

            indexed_sentence.append(idx)
        indexed_sentences.append(indexed_sentence)



  # restrict vocab size

  # set all the words I want to keep to infinity
  # so that they are included when I pick the most
  # common words
    for word in keep_words:
        word_idx_count[word2idx[word]] = float('inf')

    sorted_word_idx_count = sorted(word_idx_count.items(), key=operator.itemgetter(1), reverse=True)
    word2idx_small = {}
    new_idx = 0
    idx_new_idx_map = {}
    for idx, count in sorted_word_idx_count[:n_vocab]:
        word = idx2word[idx]
        print(word, count)
        word2idx_small[word] = new_idx
        idx_new_idx_map[idx] = new_idx
        new_idx += 1
    # let 'unknown' be the last token
    word2idx_small['UNKNOWN'] = new_idx 
    unknown = new_idx

    assert('START' in word2idx_small)
    assert('END' in word2idx_small)
    for word in keep_words:
        assert(word in word2idx_small)

    # map old idx to new idx
    sentences_small = []
    for sentence in indexed_sentences:
        if len(sentence) > 1:
            new_sentence = [idx_new_idx_map[idx] if idx in idx_new_idx_map else unknown for idx in sentence]
            sentences_small.append(new_sentence)

    return sentences_small, word2idx_small

عمل دالة لعمل مصفوفة ضخمة للبايجرام, والتي كلها بقيمة 1 في البداية , ثم يتم ضربها في معامل معين لتقليلها

ثم تحديد بداية و نهاية الكلمة 

ثم اضافة 1 الي كل كلمتين متتاليتين في مكانهما , حيث الصفوف هي الكلمات و الأعمدة هي الكلمات السابقة لها 

اخيرا  يتم عمل تدريج للبيانات لتقليلها

In [ ]:
def get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=1):
    # structure of bigram probability matrix will be:
    # (last word, current word) --> probability
    # we will use add-1 smoothing
    # note: we'll always ignore this from the END token
    bigram_probs = np.ones((V, V)) * smoothing
    for sentence in sentences:
        for i in range(len(sentence)):
            if i == 0:
                # beginning word
                bigram_probs[start_idx, sentence[i]] += 1
            else:
                # middle word
                bigram_probs[sentence[i-1], sentence[i]] += 1

            # if we're at the final word
            # we update the bigram for last -> current
            # AND current -> END token
            if i == len(sentence) - 1:
                # final word
                bigram_probs[sentence[i], end_idx] += 1

    # normalize the counts along the rows to get probabilities
    bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
    return bigram_probs


قراءة الجمل  , و قاموس الكلمات الي الاندكس من المكتبة التي تم تصميمها

In [ ]:
# load in the data
# note: sentences are already converted to sequences of word indexes
# note: you can limit the vocab size if you run out of memory
sentences, word2idx = get_sentences_with_word2idx_limit_vocab(10000)
# sentences, word2idx = get_sentences_with_word2idx()

عدد الجمل و القاموس

In [ ]:
len(sentences) , len(word2idx)

الجملة الاولي

In [ ]:
sentences[0]

فحص القاموس

In [ ]:
for i,(a,b) in enumerate(word2idx.items()) : 
    if i ==20 : break
    print(a ,'     ', b)

In [ ]:
word2idx['of']

قاموس عكسي

In [ ]:
idx2word= {a : b for b,a in word2idx.items() }

In [ ]:
for i,(a,b) in enumerate(idx2word.items()) : 
    if i ==20 : break
    print(a ,'     ', b)

In [ ]:
idx2word[20]

رؤية بعض الجمل

In [ ]:
for k in range(10) : 
    phrase = []
    for i in sentences[k] : 
        phrase.append(idx2word[i])
    print(' '.join(phrase))
    print('----------------------------------------')

تحديد حجم الكلمات لعمل المصفوفة الضخمة

In [ ]:
# vocab size
V = len(word2idx)
print("Vocab size:", V)

تحديد اندكس البداية و النهاية 

In [ ]:
# we will also treat beginning of sentence and end of sentence as bigrams
# START -> first word
# last word -> END
start_idx = word2idx['START']
end_idx = word2idx['END']
start_idx , end_idx

تطبيق الدالة السابقة

In [ ]:
# a matrix where:
# row = last word
# col = current word
# value at [row, col] = p(current word | last word)
bigram_probs = get_bigram_probs(sentences, V, start_idx, end_idx, smoothing=0.1)

حجمها

In [ ]:
bigram_probs.shape

In [ ]:
bigram_probs[0][:20]

دالة لقياس درجة الجملة

عبر حساب لوغاريتم كل كلمة علي حدة , ثم حساب المتوسط

In [ ]:
# a function to calculate normalized log prob score
# for a sentence
def get_score(sentence):
    score = 0
    for i in range(len(sentence)):
        if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, sentence[i]])
        else:
            # middle word
            score += np.log(bigram_probs[sentence[i-1], sentence[i]])
    # final word
    score += np.log(bigram_probs[sentence[-1], end_idx])

    # normalize the score
    return score / (len(sentence) + 1)



دالة لعرض الكلمات

In [ ]:
def get_words(sentence):
    return ' '.join(idx2word[i] for i in sentence)

عمل متغيرات للجمل غير الصحيحة , بحيث تكون غير متناسقة

In [ ]:
# when we sample a fake sentence, we want to ensure not to sample  start token or end token
sample_probs = np.ones(V)
sample_probs[start_idx] = 0
sample_probs[end_idx] = 0
sample_probs /= sample_probs.sum()

# test our model on real and fake sentence

الان نقوم بعرض جملة صحيحة و اخري غير صحيحة , وقياس درجة كلا منهما

ثم طلب جملة من المستخدم , والتي سنقيس درجتها

In [ ]:
while True:
    # real sentence
    real_idx = np.random.choice(len(sentences))
    real = sentences[real_idx]

    # fake sentence
    fake = np.random.choice(V, size=len(real), p=sample_probs)

    print("REAL:", get_words(real), "SCORE:", get_score(real))
    print("FAKE:", get_words(fake), "SCORE:", get_score(fake))

    # input your own sentence
    custom = input("Enter your own sentence:\n")
    custom = custom.lower().split()

    # check that all tokens exist in word2idx (otherwise, we can't get score)
    bad_sentence = False
    for token in custom:
        if token not in word2idx:
            bad_sentence = True

    if bad_sentence:
        print("Sorry, you entered words that are not in the vocabulary")
    else:
        # convert sentence into list of indexes
        custom = [word2idx[token] for token in custom]
        print("SCORE:", get_score(custom))


    cont = input("Continue? [Y/n]")
    if cont and cont.lower() in ('N', 'n'):
        break

